In [31]:
%%time
from visibility_graph import VisibilityGraph
import matplotlib.pyplot as plt
import mplleaflet

filename = "maps/kozlovo_extended.osm.pbf"
bbox = [36.25, 56.6, 36.45, 56.65]

map_data = VisibilityGraph(filename, bbox)

CPU times: user 12 s, sys: 382 ms, total: 12.4 s
Wall time: 12.9 s


In [32]:
%%time

epsilon = 0.002
bbox_comp = 10
length_comp = 20

map_data.build_dataframe(epsilon=epsilon, bbox_comp=bbox_comp, length_comp=None)

CPU times: user 1.46 s, sys: 17 µs, total: 1.46 s
Wall time: 1.54 s


In [34]:
%%time
from find_pair import *

plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = map_data.build_graph(find_pair_cutoff, add_edges_inside=True, inside_percent=0.3, graph=False, plot=plot)

# print('edges: ', G.number_of_edges())
# print('nodes: ', G.number_of_nodes())
mplleaflet.display(fig=fig)

___
find incident edges for A*

In [5]:
from find_pair import *
start = ([36.35, 56.57], None, None, None, None)
incidents = map_data.incident_vertices(start, find_pair_cutoff)
fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)